# Кибериммунный подход к разработке. Учебный пример "Светофор"

## Об авторе 

Этот блокнот разработан для вас Сергеем Соболевым, sergey.p.sobolev@kaspersky.com

Больше информации о кибериммунном подходе можно найти на странице https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82

Подписывайтесь на телеграм-канал @learning_cyberimmunity (https://t.me/learning_cyberimmunity)

Обучающие видео на тему кибериммунного подхода вы можете найти на youtube канале https://www.youtube.com/@learning_cyberimmunity/

## О примере 

Светофор - это, на первый взгляд, очень простая система, но она оказывает критическое влияние на безопасность дорожного движения. 
Применим идеи конструктивной безопасности к архитектуре и реализации прототипа светофора. За отправную точку возьмём [код базового примера](https://colab.research.google.com/github/sergey-sobolev/cyberimmune-systems-basic-demo-notebook01/blob/main/cyberimmunity-basics.ipynb) и немного его переработаем.

Для переработки будем использовать описание архитектуры решения, которое обсуждалось на занятии.

## Простая реализация выбранной политики архитектуры

Будем использовать политику архитектуры, показанную на рис. 1.

![Рис. 1. Политика архитектуры](images/tl-archpol-0.02.png)

Рис. 1. Политика архитектуры светофора

1. Создадим функциональные компоненты (сущности 1-4) и монитор безопасности, который будет контролировать их взаимодействие, в том числе реализовывать контроль конфигураций светофора (сущность №5 на архитектурной диаграмме)
2. Определим политики безопасности
3. Сымитируем запрос на изменение режима для проверки работы всех элементов

- В качестве интерфейса взаимодействия используем очереди сообщений, у каждой сущности есть своя «персональная» очередь, ассоциированная с ней
- Компоненты 1-4 отправляют сообщения только в очередь monitor сущности SecurityMonitor
- SecurityMonitor проверяет сообщения на соответствие политикам безопасности, в случае положительного решения перенаправляет сообщение в очередь соответствующей сущности

В коде назовём сущности следующим образом
1. Связь - CitySystemConnector
2. Система управления светофора - ControlSystem
3. Управление светодиодами - LightsGPIO
4. Система диагностики - SelfDiagnosticsSystem

Логику контроля режимов светофора (компонент №5 на рис. 1) реализуем в виде политик безопасности в мониторе безопасности

![Рис. 2. Политика архитектуры с именами классов](images/tl-archpol-code.png)

Рис. 2. Политика архитектуры с именами классов

Очередь событий для монитора безопасности: все запросы от сущностей друг к другу должны отправляться только в неё

In [2]:
from multiprocessing import Queue
monitor_events_queue = Queue()

Зафиксируем формат сообщений

In [3]:
from dataclasses import dataclass


@dataclass
class Event:
    source: str       # отправитель
    destination: str  # получатель
    operation: str    # чего хочет (запрашиваемое действие)
    parameters: str   # с какими параметрами

### Монитор безопасности

Ниже в методе _check_policies можно увидеть пример политики безопасности:

```python
if event.source == "ControlSystem" \
        and event.destination == "LightsGPIO" \
        and event.operation == "set_mode" \ 
        and self._check_mode(event.operation):
    authorized = True
```            

в этом примере проверяется отправитель сообщения, получатель, запрашиваемая операция и даже параметры операции. Это максимально жёсткий вариант, очевидно, в зависимости от ситуации количество проверок можно уменьшить.

А пока это место для экспериментов, как можно из монитора безопасности заблокировать взаимодействие между сущностями.

In [4]:
from multiprocessing import Queue, Process
from multiprocessing.queues import Empty
from typing import List
import json
from time import sleep

# формат управляющих команд для монитора
@dataclass
class ControlEvent:
    operation: str

# список разрешенных сочетаний сигналов светофора
# любые сочетания, отсутствующие в этом списке, запрещены
traffic_lights_allowed_configurations = [
    {"direction_1": "red", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "yellow_blinking", "direction_2": "yellow_blinking"},
    {"direction_1": "green", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "red", "direction_2": "green", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "red", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "red", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "yellow", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "off", "direction_2": "off", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "green", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "green", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},
]


class Monitor(Process):

    def __init__(self, events_q: Queue):
        super().__init__()
        self._events_q = events_q  
        self._control_q = Queue()  
        self._entity_queues = {}   
        self._command_queues: List[Queue] = []  
        self._force_quit = False   

    # регистрация очереди новой сущности
    def add_entity_queue(self, entity_id: str, queue: Queue):
        print(f"[монитор] регистрируем сущность {entity_id}")
        self._entity_queues[entity_id] = queue

    def add_entity_command_queue(self, entity_id: str, queue: Queue):
        print(f"[монитор] регистрируем очередь управляющих команд сущности {entity_id}")
        self._command_queues.append(queue)

    def _check_mode(self, mode_str: str) -> bool:
        mode_ok = False
        try:
            mode = json.loads(mode_str)
            print(f"[монитор] проверяем конфигурацию {mode}")
            if mode in traffic_lights_allowed_configurations:
                # такой режим найден, можно активировать
                mode_ok = True
        except:
            mode_ok = False
        return mode_ok

    # проверка политик безопасности        
    def _check_policies(self, event):
        print(f'[монитор] обрабатываем событие {event}')

        # default deny: всё, что не разрешено, запрещено по умолчанию!
        authorized = False

        # проверка на входе, что это экземпляр класса Event, 
        # т.е. имеет ожидаемый формат
        if not isinstance(event, Event):
            return False

        # 
        #  политики безопасности
        #

        # пример политики безопасности
        if event.source == "ControlSystem" \
                and event.destination == "LightsGPIO" \
                and event.operation == "set_mode" \
                and self._check_mode(event.parameters):
            authorized = True
    
        if event.source == "ControlSystem" \
                and event.destination == "CitySystemConnector" \
                and event.operation == "diagnostic":
            authorized = True

        if event.source == "SelfDiagnostics" \
                and event.destination == "ControlSystem" \
                and event.operation == "diagnostic":
            authorized = True

        if event.source == "LightsGPIO" \
                and event.destination == "SelfDiagnostics" \
                and event.operation == "diagnostic":
            authorized = True

        if event.source == "CitySystemConnector" \
                and event.destination == "ControlSystem" \
                and event.operation == "turn_yellow_blinking":
            authorized = True

        if event.source == "CitySystemConnector" \
                and event.destination == "ControlSystem" \
                and event.operation == "change_green_duration":
            authorized = True
            
        if event.source == "CitySystemConnector" \
                and event.destination == "ControlSystem" \
                and event.operation == "turn_default":
            authorized = True

        
        if authorized is False:
            print("[монитор] событие не разрешено политиками безопасности")
        return authorized

    def _proceed(self, event):
        print(f'[монитор] отправляем запрос {event}')
        try:
            # найдём очередь получателя события
            dst_q: Queue = self._entity_queues[event.destination]
            # и положим запрос в эту очередь
            dst_q.put(event)
        except  Exception as e:
            # например, запрос пришёл от или для неизвестной сущности
            print(f"[монитор] ошибка выполнения запроса {e}")

    # основной код работы монитора безопасности    
    def run(self):
        print('[монитор] старт')

        # в цикле проверяет наличие новых событий, 
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                event = self._events_q.get_nowait()
                authorized = self._check_policies(event)
                if authorized:
                    self._proceed(event)
            except Empty:
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[монитор] ошибка обработки {e}, {event}")
            self._check_control_q()
        print('[монитор] завершение работы')

    # запрос на остановку работы монитора безопасности для завершения работы
    # может вызываться вне процесса монитора
    def stop(self):
        # поскольку монитор работает в отдельном процессе,
        # запрос помещается в очередь, которая проверяется из процесса монитора
        request = ControlEvent(operation='stop')
        # оповещаем другие сервисы о завершении работы
        for queue in self._command_queues:
            queue.put(request)
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[монитор] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность ControlSystem

Эта сущность отправляет сообщение для другой сущности (LightsGPIO)

In [5]:
from multiprocessing import Queue, Process
import json
from time import sleep

standart_modes = [
    {"direction_1": "red", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "yellow_blinking", "direction_2": "yellow_blinking"},
    {"direction_1": "green", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "red", "direction_2": "green", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "red", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "red", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "yellow", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "off", "direction_2": "off", "direction_1_turn": "left", "direction_2_turn": "right"},
    {"direction_1": "green", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"},    
    {"direction_1": "green", "direction_2": "yellow", "direction_1_turn": "left", "direction_2_turn": "right"},
]


class ControlSystem(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        # очередь для команды выхода
        self._command_queue = Queue()

        self._switching_duration = 5 # Длительность сигнала
        self._switching_ticks = 0 # кол=во пройденных тиков основного цикла

        self._modes = standart_modes
        self._next_mode_index = 0
        self._force_quit = False

    # выдаёт собственную очередь для взаимодействия
    def entity_queue(self):
        return self._own_queue
    
    # выдает командную очередь для взаимодействия
    def command_queue(self):
        return self._command_queue
    
    # основной цикл сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')

        # главный цикл работы модуля ControlSystem
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим, 
                # чтобы можно было завершить работу монитора, 
                # не дожидаясь нового сообщения
                event: Event = self._own_queue.get_nowait()

                if event.operation == "turn_yellow_blinking":
                    print(f'[{self.__class__.__name__}] смена режима светофора на нерегулируемый')
                    self._set_yellow_blinking_mode()

                if event.operation == "turn_default":
                    if (self._modes == standart_modes):
                        print(f'[{self.__class__.__name__}] светофор уже в стандартном режиме')
                    else:
                        print(f'[{self.__class__.__name__}] смена режима светофора на стандартный')
                        self._set_standart_mode()

                if event.operation == "change_green_duration":
                    print(f'[{self.__class__.__name__}] смена длительность сигналов светофора')
                    self._switching_duration = int(event.parameters)
                
                if event.operation == "diagnostic":
                    print(f"[{self.__class__.__name__}] получены данные диагностики: {event.parameters}")
                    event = Event(source=self.__class__.__name__,
                      destination='CitySystemConnector',
                      operation='diagnostic',
                      parameters=event.parameters
                      )
                    self.monitor_queue.put(event)
                
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[{self.__class__.__name__}] ошибка обработки {e}, {event}")
            self._check_switch_duration()
            self._check_control_q()
        print(f'[{self.__class__.__name__}] завершение работы')

    def _check_switch_duration(self):
        # Отображение тиков цикла
        #print(f'[Debug] _switching_ticks: {self._switching_ticks} ; _switching_duration: {self._switching_duration}')

        if self._switching_ticks * 0.5 == self._switching_duration:
            self._do_switch()
            self._switching_ticks = 0
        else:
            self._switching_ticks += 1

    def _do_switch(self):
        if (len(self._modes) != 0):
            if self._next_mode_index >= 0 and self._next_mode_index < len(self._modes):
                next_mode = self._modes[self._next_mode_index]
                print(f'[{self.__class__.__name__}] смена режима на {next_mode}')
                event = Event(source=self.__class__.__name__,
                      destination='LightsGPIO',
                      operation='set_mode',
                      parameters=json.dumps(next_mode)
                      )
                self.monitor_queue.put(event)
                self._set_next_mode_index()
                
    
    def _set_next_mode_index(self):
        self._next_mode_index += 1

        if self._next_mode_index >= len(self._modes):
            self._next_mode_index = 0

    def _set_yellow_blinking_mode(self):
        self._modes = []
        event = Event(source=self.__class__.__name__,
                      destination='LightsGPIO',
                      operation='set_mode',
                      parameters=json.dumps({"direction_1": "yellow_blinking", "direction_2": "yellow_blinking"})
                      )
        self.monitor_queue.put(event)
    
    def _set_standart_mode(self):
        self._modes = standart_modes
    
    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._command_queue.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность SelfDiagnostics

In [6]:
from multiprocessing import Queue, Process
from time import sleep

class SelfDiagnostics(Process):
    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self._command_queue = Queue()
        self._force_quit = False

    def entity_queue(self):
        return self._own_queue
    
    # выдает командную очередь для взаимодействия
    def command_queue(self):
        return self._command_queue

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')

        # главный цикл работы модуля SelfDiagnostics
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим, 
                # чтобы можно было завершить работу монитора, 
                # не дожидаясь нового сообщения
                event: Event = self._own_queue.get_nowait()
                if event.operation == "diagnostic":
                    print(f"[{self.__class__.__name__}] диагностические данные из {event.source}: {event.parameters}")      
                    event = Event(source=self.__class__.__name__,
                      destination='ControlSystem',
                      operation='diagnostic',
                      parameters=event.parameters
                      )
                    self.monitor_queue.put(event)  
                
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[{self.__class__.__name__}] ошибка обработки {e}, {event}")
            self._check_control_q()
        print(f'[{self.__class__.__name__}] завершение работы')

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._command_queue.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность LightsGPIO

Эта сущность ждёт входящее сообщение в течение заданного периода времени, если получает - обрабатывает и завершает работу или выходит по таймауту.

In [7]:
from multiprocessing import Queue, Process
from time import sleep
import random

class LightsGPIO(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self._command_queue = Queue()
        self._force_quit = False
        self._current_mode = {"direction_1": "off", "direction_2": "off"},

    def entity_queue(self):
        return self._own_queue
    
    # выдает командную очередь для взаимодействия
    def command_queue(self):
        return self._command_queue

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')

        # главный цикл работы модуля ControlSystem
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим, 
                # чтобы можно было завершить работу монитора, 
                # не дожидаясь нового сообщения
                event: Event = self._own_queue.get_nowait()
                if event.operation == "set_mode":
                    print(f"[{self.__class__.__name__}] {event.source} запрашивает изменение режима {event.parameters}")
                    print(f"[{self.__class__.__name__}] новый режим: {event.parameters}!")            
                
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # на всякий случай, если что-то пошло не так, выведем сообщение об ошибке
                print(f"[{self.__class__.__name__}] ошибка обработки {e}, {event}")
                
            self._check_system_health() # проверка работоспособности
            self._check_control_q() # проверка наличия новых управляющих команд
        print(f'[{self.__class__.__name__}] завершение работы')

    def _check_system_health(self):
        print(f"[{self.__class__.__name__}] отправка диагностики в [SelfDiagnostics]")
        system_status = "work"
        
        if random.randint(1, 15) == 1:
            system_status = "not work"

        event = Event(source=self.__class__.__name__,
                      destination='SelfDiagnostics',
                      operation='diagnostic',
                      parameters=f'{system_status}: {self._current_mode}'
                      )
        self.monitor_queue.put(event)


    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._command_queue.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность CitySystemConnector

In [8]:
from multiprocessing import Queue, Process
import random
from time import sleep

class CitySystemConnector(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self._command_queue = Queue()
        self._force_quit = False

    # выдаёт собственную очередь для взаимодействия
    def entity_queue(self):
        return self._own_queue
    
    # выдает командную очередь для взаимодействия
    def command_queue(self):
        return self._command_queue

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        while self._force_quit is False:
            if self._force_quit is True:
                print(f'[{self.__class__.__name__}] завершение работы')
                return
            
            num = random.randint(1, 100)

            if num > 97:
                print(f'[{self.__class__.__name__}] отправляем запрос на изменение режима на нерегулируемый')
                event = Event(source=self.__class__.__name__,
                    destination='ControlSystem',
                    operation='turn_yellow_blinking',
                    parameters=""
                    )
                self.monitor_queue.put(event)
            if num < 5:
                # меняем сигналы светофора на стандартный набор, если он не уже в стандартном состоянии
                print(f'[{self.__class__.__name__}] отправляем запрос на изменение режима на стандартный')
                event = Event(source=self.__class__.__name__,
                    destination='ControlSystem',
                    operation='turn_default',
                    parameters=""
                    )
                self.monitor_queue.put(event)

                # меняем продолжительность сигнала
                duration = random.randint(3, 10)
                print(f'[{self.__class__.__name__}] отправляем запрос на изменение длительности сигнала на {duration}c')
                event = Event(source=self.__class__.__name__,
                    destination='ControlSystem',
                    operation='change_green_duration',
                    parameters=str(duration)
                    )
                self.monitor_queue.put(event)

            self._check_event_queue()
            self._check_control_q()
            
            sleep(0.5)
            
        print(f'[{self.__class__.__name__}] завершение работы')

    def _check_control_q(self):
        try:
            request: ControlEvent = self._command_queue.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

    def _check_event_queue(self):
        event = None
        try:
            # ожидание сделано неблокирующим, 
            # чтобы можно было завершить работу монитора, 
            # не дожидаясь нового сообщения
            event: Event = self._own_queue.get_nowait()
            if event.operation == "diagnostic":
                print(f"[{self.__class__.__name__}] текущее состояний светофора {event.parameters}")
        except Empty:
            # сюда попадаем, если новых сообщений ещё нет,
            # в таком случае немного подождём
            pass
        except Exception as e:
            # что-то пошло не так, выведем сообщение об ошибке
            print(f"[{self.__class__.__name__}] ошибка обработки {e}, {event}")

### Инициализируем монитор и сущности

In [9]:
monitor = Monitor(monitor_events_queue)
control_system = ControlSystem(monitor_events_queue)
lights_gpio = LightsGPIO(monitor_events_queue)
diagnostic = SelfDiagnostics(monitor_events_queue)
citySystemConnector = CitySystemConnector(monitor_events_queue)

регистрируем очереди сущностей в мониторе

In [10]:
monitor.add_entity_queue(control_system.__class__.__name__, control_system.entity_queue())
monitor.add_entity_queue(lights_gpio.__class__.__name__, lights_gpio.entity_queue())
monitor.add_entity_queue(diagnostic.__class__.__name__, diagnostic.entity_queue())
monitor.add_entity_queue(citySystemConnector.__class__.__name__, citySystemConnector.entity_queue())

monitor.add_entity_command_queue(control_system.__class__.__name__, control_system.command_queue())
monitor.add_entity_command_queue(lights_gpio.__class__.__name__, lights_gpio.command_queue())
monitor.add_entity_command_queue(diagnostic.__class__.__name__, diagnostic.command_queue())
monitor.add_entity_command_queue(citySystemConnector.__class__.__name__, citySystemConnector.command_queue())

[монитор] регистрируем сущность ControlSystem
[монитор] регистрируем сущность LightsGPIO
[монитор] регистрируем сущность SelfDiagnostics
[монитор] регистрируем сущность CitySystemConnector
[монитор] регистрируем очередь управляющих команд сущности ControlSystem
[монитор] регистрируем очередь управляющих команд сущности LightsGPIO
[монитор] регистрируем очередь управляющих команд сущности SelfDiagnostics
[монитор] регистрируем очередь управляющих команд сущности CitySystemConnector


### Запускаем всё

Ожидаемая последовательность событий

![Диаграмма последовательности вызовов](https://www.plantuml.com/plantuml/png/dPBVIiCm6CNlynIvdtk1NSZ02n4KXJr1w886-cUqcR0xgoBpIdoJLgqhtRg-mfStygJPM3js9OLyoPTpVia97ITQn7eU-6o6gZmr4w7c5r6euyYVB18j0ouIxhb6JpIHtZnMUd4JXKf7iPK5RjgJNQlx1vrStbtTMeNVhXZR0VdmV6yQ34QSQjhI5sNcWrE5QMrUgQHlysAUq7oZqcxyq1hbW6KxG8S5KWEBPHMe5MLeOBa6uHd4YWbVSs1JQg1OKktEF3ATSTI2Vk7Os6b6AupGerctn3uM5WWfn_OAxGRGr4OogTtktjCzmt3utyZ2q-fHQBb_JrSEv177oHigRB1E2ChOL1vxfPz8ZWjdBhv9ELn5Bw_vfFf4L2fFlZsEtkBBpNjxWH8mkyHWbbZbC1TCXbCsne1Vxmy0)

In [11]:
citySystemConnector.start()

[CitySystemConnector] старт
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора not work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее состояний светофора work: ({'direction_1': 'off', 'direction_2': 'off'},)
[CitySystemConnector] текущее со

In [12]:
monitor.start()

[монитор] старт
[монитор] обрабатываем событие Event(source='LightsGPIO', destination='SelfDiagnostics', operation='diagnostic', parameters="work: ({'direction_1': 'off', 'direction_2': 'off'},)")
[монитор] отправляем запрос Event(source='LightsGPIO', destination='SelfDiagnostics', operation='diagnostic', parameters="work: ({'direction_1': 'off', 'direction_2': 'off'},)")
[монитор] обрабатываем событие Event(source='LightsGPIO', destination='SelfDiagnostics', operation='diagnostic', parameters="work: ({'direction_1': 'off', 'direction_2': 'off'},)")
[монитор] отправляем запрос Event(source='LightsGPIO', destination='SelfDiagnostics', operation='diagnostic', parameters="work: ({'direction_1': 'off', 'direction_2': 'off'},)")
[монитор] обрабатываем событие Event(source='SelfDiagnostics', destination='ControlSystem', operation='diagnostic', parameters="work: ({'direction_1': 'off', 'direction_2': 'off'},)")
[монитор] отправляем запрос Event(source='SelfDiagnostics', destination='ControlSy

In [13]:
control_system.start()

[ControlSystem] старт
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] смена режима на {'direction_1': 'red', 'direction_2': 'red', 'direction_1_turn': 'left', 'direction_2_turn': 'right'}
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: not work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[ControlSystem] получены данные диагностики: wor

In [14]:
lights_gpio.start()

[LightsGPIO] старт
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] ControlSystem запрашивает изменение режима {"direction_1": "red", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"}
[LightsGPIO] новый режим: {"direction_1": "red", "direction_2": "red", "direction_1_turn": "left", "direction_2_turn": "right"}!
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagnostics]
[LightsGPIO] отправка диагностики в [SelfDiagn

In [15]:
diagnostic.start()

[SelfDiagnostics] старт
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: not work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direction_2': 'off'},)
[SelfDiagnostics] диагностические данные из LightsGPIO: work: ({'direction_1': 'off', 'direc

In [16]:
sleep(30)

### Теперь останавливаем

In [18]:
monitor.stop()
control_system.join()
lights_gpio.join()
diagnostic.join()
citySystemConnector.join()
monitor.join()

## Заключение

В этом блокноте продемонстрирован базовый функционал контролируемого изменения режима работы светофора. 

В примере не реализованы некоторые сущности и большая часть логики работы светофора, которую можно предположить по архитектурной диаграмме. Попробуйте сделать это самостоятельно!

## Упражнения

Уровень "Новичок"

- в коде ControlSystem измените режим на недопустимый (два зелёных) и выполните все ячейки. Убедитесь, что монитор безопасности заблокировал сообщение, как нарушающее политику безопасности
- измените политики безопасности так, чтобы был возможен режим "моргающий жёлтый" (yellow_blinking), переводящий перекрёсток в режим нерегулируемого

Уровень "Средней сложности"

- добавьте политики безопасности для доп. секций со стрелками (поворот налево или направо)
- измените код сущностей, чтобы они не завершали работу после одного сообщения, а работали произвольное время (см. реализацию монитора безопасности)
- реализуйте сущность само-диагностики (SelfDiagnostics) и отправку сообщений от LightsGPIO (необходимо доработать политики безопасности!)

Уровень "Продвинутый"

- измените код сущности ControlSystem, реализуйте смену режимов по таймеру (заданную длительность зелёного по каждому направлению)
- реализуйте сущность CitySystemConnector, которая будет имитировать получение изменения режима, реализуйте взаимодействие CitySystemConnector и ControlSystem (понадобится доработать политики безопасности). Например, изменение длительности зелёного по направлениям или отключение светофора (перевод перекрёстка в режим нерегулируемого)
- реализуйте передачу в компонент CitySystemConnector информации об исправности светофора (статус самодиагностики; текущий режим работы). В компоненте реализуйте вывод в виде сообщений о состоянии системы